In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
import kinect
import cv2
from scipy.signal import find_peaks

dir_path = "D:\\HK_PS4_Data to Xuan & Junjie\\Kinect_Sensor_data"

In [3]:
#  获得所有测试的Kinect文件路径，并计数
kinect_files_path = kinect.get_file_path(dir_path)

In [4]:
imu_file_path = "E:\\JupyterLab\\TUG\\IMU_time.xlsx"
tps_dict = kinect.kin_imutps_dict(imu_file_path, kinect_files_path)

In [8]:
gait_feature_dict = {}

for i in range(len(kinect_files_path)):
    kinect_data = kinect.Kinect(pd.read_csv(kinect_files_path[i]))
    
    #  找到转身的两个点 划出原始数据的两个时间段，画图
    time_stamp = kinect_data.get_timestamp()
    tps_df = tps_dict[kinect_files_path[i]][2]
    imu_walking_start = tps_df.iloc[0,0]
    imu_turning_start = tps_df.iloc[0,1]
    imu_turning_end = tps_df.iloc[0,2]
    imu_walking_end = tps_df.iloc[0,3]
    
    #  在Kinect数据点中寻找与IMU切分点最接近的点
    walking_start = kinect.find_nearest(time_stamp, imu_walking_start)
    turning_start = kinect.find_nearest(time_stamp, imu_turning_start)
    turning_end = kinect.find_nearest(time_stamp, imu_turning_end)
    walking_end = kinect.find_nearest(time_stamp, imu_walking_end)
    
    walking_time = np.round((time_stamp[turning_start]-time_stamp[walking_start])+(time_stamp[walking_end]-time_stamp[turning_end]), 2)
    
    centre_name, test_id, test_date = kinect.get_test_info(kinect_files_path[i])
    
    gait_feature_dict[kinect_files_path[i]] = [centre_name, test_id, test_date, walking_time]

gait_feature = pd.DataFrame(gait_feature_dict)
gait_feature_t = pd.DataFrame(gait_feature.values.T, index=gait_feature.columns, columns=gait_feature.index)
gait_feature_t.to_excel('./walking_time.xlsx')